<img src="assets/banner.png" width="100%" align="left" style="left">

## RF Automatic Gain Control with PYNQ

We present an Automatic Gain Control (AGC) circuit for control of an analogue/RF amplifier with an interactive GUI exposing its parameters. This example requires an external amplifier (the [VeGA from Nooelec](https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf)) and some attenuators.
This very simple design focuses on using the RFSoC's "threshold" feedback signals, allowing us to make decisions based on the true input signal --- even before any mixing or decimation. This let's us ensure that the input signal is occupying the full scale of the ADC, minimising quantisation errors without exceeding the limits of the device. We'll generate various input signals, explore some of the limits of this  AGC algorithm, and practice tweaking our parameters for best performance.
The design of this AGC example and the use of the RF Data Converter's threshold signals will be explored.

## Aims
* Set up an AGC system with an RFSoC and an external amplifier
* Design test signals and capture our system's output at full RF speeds
* Interactively explore the behaviour of an AGC circuit and amplifier
* Learn to tune the AGC parameters for different scenarios and encounter the limits of our amplifier configuration

## Table of Contents

* [1. Introduction](#1.-Introduction)
* [2. ...And we're off!](#go)
* [3. Exploring some scenarios](#scenarios)
* [4. A closer look at the hardware](#hw)
    * [4.1 Power estimation](#power-est)
    * [4.2 Logarithms and exponentials](#log-est)
* [5. Conclusion](#conclusion)

----

## 1. Introduction

Unlike the previous example, our FPGA design will be quite straightforward here. We will rely on the RF Data Converter for our signal monitoring and an [external amplifier](https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf) to provide analogue gain.
This means that the system is suitable for automatic gain control of RF signals _before_ they enter the ADC, rather than the previous example's fully digital approach for baseband.
Working in the analogue world means that our goals are a little different too. Instead of providing a constant average power for downstream synchronistation or receiver circuits, we will focus more on matching the RF signal to the characteristics of the ADC. In short, we want to ensure that our signal of interest occupies as much of the ADC's range as possible, without exceeding its upper limit. This will help us minimise quantisation errors and give the downstream design a signal with the best characteristics we can.

Let's take a look at how this hardware design is structured.

<a class="anchor" id="fig-1"></a>
<figure>
    <img src='./assets/aagc_loopback_overview.svg'/>
    <figcaption><b>Figure 1: System overview </b></figcaption>
</figure>

This version of our AGC example introduces the RF Data Converter block and an external amplifier. Apart from these additions, the overall structure remains quite familiar. We'll use PYNQ to craft a test signal in software, send it to the DAC via a DMA channel, and simultaneously receive from the ADC on another DMA channel.

Just before we start to interact with this design, let's take a moment to explore the AGC circuit and the assistance we get from the Data Converter's threshold signals.

### Using the RF Data Converter's thresholds

The ADCs on the RF Data Converter can provide us with two user controlled threshold flags per channel. Note there are other flags for overrange and overvoltage events, but we simply ignore them here.

For our purposes, we'll employ the `hysteresis` mode. Full documentation for all modes can be found in Xilinx's [Product Guide 269](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_4/pg269-rf-data-converter.pdf). Hysteresis mode allows us to configure an upper threshold value, a lower threshold value, and a hysteresis time. The output will be set as soon as the analogue input to the ADC exceeds our upper threshold value. It will remain set until the input signal remains below our lower threshold for our entire hysteresis time.

WE USE TWO THRESHOLDS WITH ONE VALUE EACH.

The plot below shows a threshold signal in hysteresis mode with an example signal with AGC running.

<a class="anchor" id="fig-1"></a>
<figure>
    <img src='./assets/rfdc_threshold_graph.svg'/>
    <figcaption><b>Figure 2: Example of Hysteresis thresholding in action </b></figcaption>
</figure>


In [ ]:
import pynq_agc
from pynq import Overlay
import plotly.express as px

ol = Overlay('block_design.bit')

In [ ]:
ol.rf_agc.gui()